# Deep Research: ETF Pairs Trading Optimization

## Objective
Maximize Sharpe ratio for the ETF Pairs Trading strategy.

## Strategy Overview
- **Pair Selection**: Cointegration test + half-life filter + volatility filter
- **Entry**: Spread deviates from mean (z-score > threshold)
- **Exit**: Spread reverts to mean (z-score crosses 0)
- **Risk Management**: Per-leg stops disabled (preserve market neutrality)

## Current Parameters (to be optimized)
- `half_life_max`: 30 days (filter slow-reverting pairs)
- `z_exit_threshold`: 0.0 (exit at mean)
- `stop_loss_per_leg`: False

## Research Questions
1. What is the optimal half-life threshold for pair selection?
2. Should we use a z-score exit threshold other than 0?
3. What spread volatility threshold maximizes risk-adjusted returns?
4. Which ETF pairs have the most stable cointegration relationship?

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
from itertools import combinations

# ETF Universe
ETF_LIST = [
    'SPY',   # S&P 500
    'QQQ',   # NASDAQ-100
    'IWM',   # Russell 2000
    'DIA',   # Dow Jones
    'VTI',   # Total Market
    'XLE',   # Energy
    'XLK',   # Technology
    'XLF',   # Financial
    'XLV',   # Healthcare
    'XLY',   # Consumer Discretionary
]

print(f"ETF Universe: {len(ETF_LIST)} ETFs")
print(f"Possible pairs: {len(list(combinations(ETF_LIST, 2)))}")

In [ ]:
# Download ETF data
print("Downloading ETF data...")
etf_data = {}
for etf in ETF_LIST:
    try:
        df = yf.download(etf, start="2015-01-01", end="2025-02-18", progress=False)
        if not df.empty:
            etf_data[etf] = df['Close'].copy()
            if isinstance(etf_data[etf], pd.DataFrame):
                etf_data[etf] = etf_data[etf].iloc[:, 0]
            print(f"  {etf}: {len(etf_data[etf])} days")
    except Exception as e:
        print(f"  {etf}: Error - {e}")

print(f"\nSuccessfully downloaded {len(etf_data)} ETFs")

In [ ]:
# Calculate half-life of mean reversion
def calculate_half_life(spread):
    """Calculate half-life of mean reversion using OU process"""
    spread_lagged = spread[:-1]
    spread_current = spread[1:]
    
    # Remove NaN
    mask = ~(np.isnan(spread_lagged) | np.isnan(spread_current))
    spread_lagged = spread_lagged[mask]
    spread_current = spread_current[mask]
    
    if len(spread_lagged) < 10:
        return 999
    
    # Calculate lag-1 autocorrelation
    corr_lag1 = np.corrcoef(spread_lagged, spread_current)[0, 1]
    
    if np.isnan(corr_lag1) or corr_lag1 <= 0 or corr_lag1 >= 1:
        return 999
    
    # Half-life = -ln(2) / ln(corr)
    half_life = -np.log(2) / np.log(corr_lag1)
    return half_life

# Test with example pair (SPY-QQQ)
if 'SPY' in etf_data and 'QQQ' in etf_data:
    # Align data
    df = pd.DataFrame({'SPY': etf_data['SPY'], 'QQQ': etf_data['QQQ']}).dropna()
    
    # Calculate spread (price ratio)
    spread = df['SPY'] / df['QQQ']
    
    # Calculate half-life
    hl = calculate_half_life(spread.values)
    print(f"SPY-QQQ Half-life: {hl:.1f} days")
    
    # Calculate spread volatility
    spread_vol = spread.pct_change().std() * np.sqrt(252)
    print(f"SPY-QQQ Spread Volatility: {spread_vol*100:.2f}%")

In [ ]:
# Select pairs based on cointegration and half-life
def select_pairs(etf_data, 
                 pvalue_threshold=0.05,
                 vol_threshold=0.01,
                 half_life_max=30):
    """
    Select cointegrated pairs with:
    - P-value < pvalue_threshold (strong cointegration)
    - Spread volatility > vol_threshold (sufficient deviation)
    - Half-life < half_life_max (reasonably fast reversion)
    """
    results = []
    
    for etf1, etf2 in combinations(etf_data.keys(), 2):
        # Align data
        df = pd.DataFrame({etf1: etf_data[etf1], etf2: etf_data[etf2]}).dropna()
        
        if len(df) < 252:
            continue
        
        try:
            # Cointegration test
            score, pvalue, _ = coint(df[etf1], df[etf2])
            
            # Calculate spread (hedge-adjusted)
            spread = df[etf1] - df[etf2] * (df[etf1].mean() / df[etf2].mean())
            
            # Spread volatility
            vol = spread.pct_change().std()
            
            # Half-life
            hl = calculate_half_life(spread.values)
            
            # Correlation
            corr = df[etf1].corr(df[etf2])
            
            if pvalue < pvalue_threshold and vol > vol_threshold and hl < half_life_max:
                results.append({
                    'etf1': etf1,
                    'etf2': etf2,
                    'pvalue': pvalue,
                    'correlation': corr,
                    'volatility': vol,
                    'half_life': hl
                })
        except Exception as e:
            continue
    
    return pd.DataFrame(results)

# Test current parameters
pairs_current = select_pairs(etf_data, half_life_max=30)
print(f"Pairs with half_life < 30 days: {len(pairs_current)}")
if not pairs_current.empty:
    print(pairs_current.sort_values('half_life').head(10).to_string())

In [ ]:
# Backtest pairs trading strategy
def backtest_pairs(etf_data, pairs, z_entry=2.0, z_exit=0.0, lookback=20):
    """
    Simple backtest for pairs trading.
    
    Entry: When z-score > z_entry (or < -z_entry)
    Exit: When z-score crosses z_exit
    """
    if pairs.empty:
        return None
    
    total_returns = []
    
    for _, row in pairs.iterrows():
        etf1, etf2 = row['etf1'], row['etf2']
        
        # Get aligned data
        df = pd.DataFrame({etf1: etf_data[etf1], etf2: etf_data[etf2]}).dropna()
        
        # Calculate spread
        hedge_ratio = (df[etf1].mean() / df[etf2].mean())
        spread = df[etf1] - df[etf2] * hedge_ratio
        
        # Z-score
        spread_mean = spread.rolling(lookback).mean()
        spread_std = spread.rolling(lookback).std()
        z_score = (spread - spread_mean) / spread_std
        
        # Generate signals
        long_signal = (z_score > z_entry).astype(int) - (z_score < -z_entry).astype(int)
        
        # Exit when z_score crosses z_exit
        exit_long = (z_score <= z_exit) & (long_signal == 1)
        exit_short = (z_score >= -z_exit) & (long_signal == -1)
        
        # Calculate returns (simplified)
        ret1 = df[etf1].pct_change()
        ret2 = df[etf2].pct_change()
        
        # Pair return (long ETF1, short ETF2)
        pair_ret = ret1 - hedge_ratio * ret2
        
        # Strategy return
        strat_ret = pair_ret * long_signal.shift(1)
        total_returns.append(strat_ret)
    
    # Combine all pair returns
    combined = pd.concat(total_returns, axis=1).mean(axis=1)
    combined = combined.dropna()
    
    # Metrics
    sharpe = np.sqrt(252) * combined.mean() / combined.std() if combined.std() > 0 else 0
    total_return = (1 + combined).cumsum().iloc[-1]
    max_dd = (combined.cumsum() / combined.cumsum().cummax() - 1).min()
    
    return {
        'sharpe': sharpe,
        'total_return': total_return,
        'max_drawdown': max_dd
    }

# Test current parameters
result = backtest_pairs(etf_data, pairs_current, z_entry=2.0, z_exit=0.0)
if result:
    print("Current Parameters (half_life_max=30, z_exit=0.0):")
    print(f"  Sharpe: {result['sharpe']:.3f}")
    print(f"  Total Return: {result['total_return']*100:.1f}%")
    print(f"  Max Drawdown: {result['max_drawdown']*100:.1f}%")

In [ ]:
# Grid search for optimal parameters
def grid_search_pairs(etf_data):
    results = []
    
    for hl_max in [15, 20, 25, 30, 40, 50]:
        # Select pairs with this half-life threshold
        pairs = select_pairs(etf_data, half_life_max=hl_max)
        
        if pairs.empty:
            continue
        
        for z_exit in [-0.5, 0.0, 0.5, 1.0]:
            for lookback in [10, 20, 30, 60]:
                result = backtest_pairs(etf_data, pairs, z_entry=2.0, z_exit=z_exit, lookback=lookback)
                
                if result:
                    results.append({
                        'half_life_max': hl_max,
                        'z_exit': z_exit,
                        'lookback': lookback,
                        'num_pairs': len(pairs),
                        'sharpe': result['sharpe'],
                        'total_return': result['total_return'],
                        'max_drawdown': result['max_drawdown']
                    })
    
    return pd.DataFrame(results)

print("Running grid search...")
grid_results = grid_search_pairs(etf_data)
grid_results = grid_results.sort_values('sharpe', ascending=False)

print("\nTop 10 Parameter Combinations:")
print(grid_results.head(10).to_string())

## Research Findings & Recommendations

### Analysis Summary
Based on the grid search:

1. **Optimal Half-Life**: Pairs with faster reversion tend to be more reliable
2. **Z-Exit**: A small positive z_exit (0.5) may reduce whipsaw
3. **Lookback Period**: Shorter lookbacks (10-20) are more responsive

### Recommended Parameters

```python
# In main.py
HALF_LIFE_MAX = <VALUE>  # From grid search
Z_EXIT_THRESHOLD = <VALUE>  # From grid search  
LOOKBACK_PERIOD = <VALUE>  # From grid search
```